# World Bank Data Cleaner

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

## WB Data converter:  

Uploading the first WB feature's csv: 

In [37]:
gdp_g = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/GDP_growth.csv')
gdp_g.head()

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Last Updated Date,2020-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.0,2019.0,NaN
4,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,3.446055,-1.369863,4.198232,0.848228,-0.450586,-0.211225,1.330511,NaN,NaN,NaN


Devising a custom cleaning function to then be applied to all WB features: after checking, they share nearly the same format so they can be adapted in a similar way.

After first inspection, I realise the below countries need renaming to match IRENA's notations:

In [ ]:
countries = {'United States' : 'USA', 'United Kingdom' : 'UK', 'Korea, Rep.': 'Korea DPR'}

In [2]:
def country_conv(x):
    countries = {'United States' : 'USA', 'United Kingdom' : 'UK', 'Korea, Rep.': 'Korea DPR'}
    for i, c in countries.items():
        if x == i:
            return c
        else:
            pass

In [2]:
#NE to clean DataFrames extracted from the WB. Below, I test the required steps to build a function from: 
#Drop 1st 3 rows & last column, turn years into integers, rename columns headers, pivot, sort & rename & drop 
#g, 
def country_conv(x):
    countries = {'United States' : 'USA', 'United Kingdom' : 'UK', 'Korea, Rep.': 'Korea DPR'}
    for i, c in countries.items():
        if x == i:
            return c
        else:
            pass

def wb_cleaner(df):
    df = df.iloc[3:, :-1]
    df.drop('Unnamed: 3', axis=1, inplace=True)
    df.iloc[0,3:] = pd.to_datetime(df.iloc[0,3:].apply(lambda x: int(x)), format = '%Y')
    df.rename(columns = df.iloc[0], inplace=True)
    
    df = df.melt(id_vars=list(df.columns[:3]),
            var_name='Date',
            value_name ='Value')
    
    df.sort_values(by = ['Country Name', 'Date'], inplace=True)
    measure = df['Indicator Name'][1]
    df.rename(columns = {'Country Name': 'Country', 'Value' : measure}, inplace=True)
    df.drop(['Country Code', 'Indicator Name'], axis =1, inplace=True)
    df = df.dropna()
    return df

## Converting WB Data:  

Below, I process every new feature's csv into an adequate format, mergeable with the master frame (IRENA's)

Natural Resource Rent:

In [1]:
nat_rent = pd.read_csv('/Users/valentinereltien/Desktop/NatRent.csv')

NameError: name 'pd' is not defined

In [32]:
NtlRent = wb_cleaner(nat_rent)

Total Area covered by Forest:

In [33]:
forest_area = pd.read_csv('/Users/valentinereltien/Desktop/ForestArea.csv')

In [34]:
ForestA = wb_cleaner(forest_area)

In [35]:
ForestA.shape, NtlRent.shape

((6917, 3), (10606, 3))

The two above features were added upon the second running of this model. I therefore merged only the two of these as they were then added to the already completed masterframe used in the first round. I include basic cleaning: reducing the years to observations post 2000 and transform the date to a datetime format, in accordance with IRENA's. I conclude by saving the feature to an independent csv file, in case I needed to use it separately. 

In [25]:
envt_merged = pd.DataFrame()

In [26]:
envt_merged = pd.merge(ForestA, NtlRent, on = ['Country', 'Date'], how= 'left')

In [27]:
pd.to_datetime(envt_merged.Date, format = '%Y').dt.year

0       1990
1       1991
2       1992
3       1993
4       1994
        ... 
6912    2012
6913    2013
6914    2014
6915    2015
6916    2016
Name: Date, Length: 6917, dtype: int64

In [28]:
envt_merged.Date = pd.to_datetime(envt_merged.Date, format = '%Y').dt.year

In [29]:
envmt = envt_merged[envt_merged.Date >= 2000]

In [36]:
envmt.shape

(4354, 4)

In [30]:
envmt.to_csv('Add_envt.csv', index=False)

Adding Net National Adjusted savings: 

In [6]:
adjsavs_usd = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/adjusted_sav_CL.csv')

In [5]:
Adjsav_usd = wb_cleaner(adjsavs_usd)

In [8]:
Adjsav_usd.to_csv('                                            ', index=False)

In [ ]:
gdp.to_csv('GDP.csv', index=False)

Merging first two features together, first two bricks of the aimed-for WB master dataframe: merging all features together before a final merge onto IRENA's: 

In [ ]:
master_lefted = pd.merge(gdp, adj_savings, on = ['Country', 'Date'], how = 'left')

Done with adjusted savings:  

In [ ]:
adj_sav = pd.read_csv('/Users/valentinereltien/Desktop/WBank/WB_GDP/Adjusted_savings.csv')

In [ ]:
adj_savings = wb_cleaner(adj_sav)

In [ ]:
adj_savings.to_csv('adjusted_sav_CL.csv', index=False)

Adding, cleaning & merging GNP feature:

In [ ]:
gnp = pd.read_csv('/Users/valentinereltien/Desktop/WBank/GNP.csv')

In [ ]:
gnp = wb_cleaner(gnp)

In [ ]:
gnp.to_csv('GNP_Cl.csv', index=False)

In [ ]:
master_lefted = pd.merge(master_lefted, gnp, on = ['Country', 'Date'], how = 'left')

Adjusted National Savings including Energy Depletion:

In [ ]:
nrj_dep = pd.read_csv('/Users/valentinereltien/Desktop/WBank/Energy_depletion.csv')

In [ ]:
nrj_dep = wb_cleaner(nrj_dep)

In [ ]:
nrj_dep.to_csv('energy_dep.csv', index=False)

In [ ]:
WB_lefted = pd.merge(master_lefted, nrj_dep, on = ['Country', 'Date'], how = 'left')

Adjusted National Savings including CO2 damage:

In [39]:
CO2_dmg = pd.read_csv('/Users/valentinereltien/Desktop/WBank/CO2_dam.csv')

In [40]:
CO2 = wb_cleaner(CO2_dmg)

In [41]:
CO2.head()

,Country,Date,Adjusted savings: carbon dioxide damage (current US$)
2652,Afghanistan,1970-01-01,2.77456e+06
2917,Afghanistan,1971-01-01,3.39641e+06
3182,Afghanistan,1972-01-01,2.93962e+06
3447,Afghanistan,1973-01-01,3.42623e+06
3712,Afghanistan,1974-01-01,4.50012e+06


In [ ]:
WB_lefted = pd.merge(WB_lefted, CO2, on = ['Country', 'Date'], how = 'left')

In [42]:
CO2.to_csv('CO2_wb.csv', index=False)

Adjusted National Savings including Particulate Emission:

In [ ]:
pt_ems = pd.read_csv('/Users/valentinereltien/Desktop/WBank/Particulate_ems.csv')

In [ ]:
pt_ems = wb_cleaner(pt_ems)

In [ ]:
pt_ems.to_csv('Ptc_em.csv', index=False)

In [ ]:
WB_lefted = pd.merge(WB_lefted, pt_ems, on = ['Country', 'Date'], how = 'left')

In [ ]:
WB_lefted.head()

In [ ]:
WB_lefted.to_csv('WB_indicators_2.csv', index=False)

In [ ]:
WB = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/WB_indicators_2.csv')
WB.head()

Attempt at adding Private Investment, I eventually retract it as it contains too few observations: 

In [ ]:
private_inv = pd.read_csv('/Users/valentinereltien/Desktop/WBank/WB_privateinv.csv')
private_inv.head()

In [ ]:
pvinv = wb_cleaner(private_inv)

In [ ]:
pvinv.to_csv('WB_PrvREinv.csv', index=False)

Adding, cleaning & merging the Unemployment Rate: 

In [26]:
unemp = pd.read_csv('/Users/valentinereltien/Desktop/WBank/WB_unemp.csv')

In [28]:
unemp = wb_cleaner(unemp)

In [29]:
unemp.to_csv('WB_unemp_cl.csv', index=False)

After re-opening this notebook, I reloading each individual feature to merge them together in a single loop, ensuring the process' uniformity: 

In [3]:
adj_savings = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/adjusted_sav_CL.csv')

In [4]:
gdp = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/GDP.csv')

In [5]:
gnp = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/GNP_Cl.csv')

In [6]:
nrj_dep = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/energy_dep.csv')

In [7]:
CO2 = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/CO2_wb.csv')

In [8]:
pt_ems = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/Ptc_em.csv')

In [9]:
pvinv = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/WB_PrvREinv.csv')

In [10]:
unemp = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/WB_unemp_cl.csv')
unemp.head()

,Country,Date,"Unemployment, total (% of total labor force) (modeled ILO estimate)"
0,Afghanistan,1991-01-01,11.2180004119873
1,Afghanistan,1992-01-01,11.2930002212524
2,Afghanistan,1993-01-01,11.4420003890991
3,Afghanistan,1994-01-01,11.470000267028801
4,Afghanistan,1995-01-01,11.4770002365112


In [10]:
CO2.head()

,Country,Date,Adjusted savings: carbon dioxide damage (current US$)
0,Afghanistan,1970-01-01,2774563.82969348
1,Afghanistan,1971-01-01,3396414.60259953
2,Afghanistan,1972-01-01,2939621.23541783
3,Afghanistan,1973-01-01,3426233.12117702
4,Afghanistan,1974-01-01,4500115.51485246


I create a list of all features in order to loop through them in a merging function for the complete WB set:

In [11]:
WB_dfs = [adj_savings, gnp, nrj_dep, CO2, pt_ems, pvinv, unemp, co2e]

In [12]:
df_m = gdp.copy()

In [13]:
pd.merge(gdp, adj_savings, on = ['Country', 'Date'], how = 'outer').merge(gnp)

,Country,Date,GDP growth (annual %),Adjusted savings: gross savings (% of GNI),GNI growth (annual %)
0,Albania,2009-01-01,3.35006739254982,17.56145981,1.30684585245906
1,Albania,2010-01-01,3.70689143341055,20.17123356,4.3096494698954
2,Albania,2011-01-01,2.54532265145231,20.53417201,3.6098457178355403
3,Albania,2012-01-01,1.41752440769345,19.77571852,0.6519218056270261
4,Albania,2013-01-01,1.00198870924962,17.43979963,3.45312621040412
...,...,...,...,...,...
6996,Zimbabwe,2014-01-01,2.37692932698003,-3.350498743,2.6741109172818702
6997,Zimbabwe,2015-01-01,1.7798727034029902,-9.108389096,0.18180389096691799
6998,Zimbabwe,2016-01-01,0.7558692509306009,-1.3341581470000001,2.4687219431414
6999,Zimbabwe,2017-01-01,4.70403534833559,-2.423258669,4.744133314800241


Simple merging loop: 

In [14]:
for df in WB_dfs:
    df_m= pd.merge(df_m, df, on = ['Country', 'Date'], how = 'left')
print(df_m.columns)

Index(['Country', 'Date', 'GDP growth (annual %)',
       'Adjusted savings: gross savings (% of GNI)', 'GNI growth (annual %)',
       'Adjusted savings: energy depletion (current US$)',
       'Adjusted savings: carbon dioxide damage (current US$)',
       'Adjusted savings: particulate emission damage (% of GNI)',
       'Investment in energy with private participation (current US$)',
       'Unemployment, total (% of total labor force) (modeled ILO estimate)',
       'CO2 emissions (metric tons per capita)'],
      dtype='object')


Copying the above frame for some basic EDA, thus leaving the origial unchanged:

In [22]:
Wb_data = df_m.copy()
Wb_data.sort_values(by='Date').Date.value_counts()

2003-01-01    251
2004-01-01    251
2011-01-01    251
2012-01-01    251
2007-01-01    251
2006-01-01    251
2013-01-01    251
2009-01-01    251
2010-01-01    251
2005-01-01    251
2014-01-01    251
2008-01-01    250
2015-01-01    250
2016-01-01    249
2017-01-01    249
2002-01-01    246
2001-01-01    245
2000-01-01    240
2018-01-01    240
1999-01-01    239
1998-01-01    237
1996-01-01    234
1997-01-01    234
1995-01-01    228
1994-01-01    225
1993-01-01    223
1992-01-01    220
1991-01-01    219
1990-01-01    208
1989-01-01    205
1988-01-01    202
1987-01-01    199
1986-01-01    194
1985-01-01    193
1984-01-01    190
1983-01-01    190
1982-01-01    187
1981-01-01    182
1980-01-01    171
1978-01-01    170
1979-01-01    170
1977-01-01    165
1976-01-01    164
1975-01-01    160
1973-01-01    158
1971-01-01    158
1974-01-01    158
1972-01-01    158
1970-01-01    144
1969-01-01    144
1968-01-01    140
1967-01-01    139
1966-01-01    135
1963-01-01    127
1965-01-01    127
1964-01-01

In [16]:
type(Wb_data.Date[0])

str

In [17]:
Wb_data.Date = pd.to_datetime(Wb_data.Date)

In [18]:
Wb_data.to_csv('Wb_stats.csv', index=False)

This feature (level of CO2 emissions) was uploaded subsequently, in an attempt to improve the model's performance. Didn't add it to the master frame due to its majority of missing values:

In [82]:
co2e_cp2000s = co2e_cp[co2e_cp.Date > '2000-01-01']

In [24]:
Wb_data[Wb_data['Date'] >= '2000-01-01']

,Country,Date,GDP growth (annual %),Adjusted savings: gross savings (% of GNI),GNI growth (annual %),Adjusted savings: energy depletion (current US$),Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: particulate emission damage (% of GNI),Investment in energy with private participation (current US$),"Unemployment, total (% of total labor force) (modeled ILO estimate)",CO2 emissions (metric tons per capita)
0,Afghanistan,2003-01-01,8.83227780370231,NaN,NaN,958155.3278581591,19983570.0004124,NaN,1580000.0,11.604000091552699,0.0504813357583004
1,Afghanistan,2004-01-01,1.4141179802191601,NaN,NaN,1244751.78551264,16861248.694275,NaN,NaN,11.5279998779297,0.0384100431744103
2,Afghanistan,2005-01-01,11.2297148388595,NaN,NaN,1472570.39904863,24954809.424183603,NaN,NaN,11.4289999008179,0.0517439645638815
3,Afghanistan,2006-01-01,5.35740324754602,NaN,NaN,1469932.27544724,32986763.503094103,NaN,NaN,11.2239999771118,0.0624275315344817
4,Afghanistan,2007-01-01,13.8263195390595,NaN,NaN,3237604.3216862003,47862356.5078225,NaN,NaN,11.045999526977502,0.0838928056625891
...,...,...,...,...,...,...,...,...,...,...,...
11703,Zimbabwe,2014-01-01,2.37692932698003,-3.350498743,2.6741109172818702,49173804.6532692,350109495.145631,0.889980093776045,NaN,5.34200000762939,0.8847212943322951
11704,Zimbabwe,2015-01-01,1.7798727034029902,-9.108389096,0.18180389096691799,28443627.567908503,353528790.600832,0.9053993732324008,NaN,5.30200004577637,NaN
11705,Zimbabwe,2016-01-01,0.7558692509306009,-1.3341581470000001,2.4687219431414,13479167.2791364,353557473.900404,0.8950909701813691,NaN,5.25299978256226,NaN
11706,Zimbabwe,2017-01-01,4.70403534833559,-2.423258669,4.744133314800241,31322783.559058197,373032248.26923996,0.845351445542448,65700000.0,5.1680002212524405,NaN


In [61]:
for c in Wb_data.columns:
    print ("---- %s ---" % c)
    print (Wb_data[c].value_counts())

---- Country ---
Country Name                 60
Chile                        58
Nigeria                      58
Central African Republic     58
Luxembourg                   58
                             ..
Channel Islands               9
Sint Maarten (Dutch part)     8
Turks and Caicos Islands      7
South Sudan                   7
Djibouti                      5
Name: Country, Length: 259, dtype: int64
---- Date ---
2003-01-01    251
2007-01-01    251
2004-01-01    251
2009-01-01    251
2012-01-01    251
2006-01-01    251
2010-01-01    251
2014-01-01    251
2005-01-01    251
2011-01-01    251
2013-01-01    251
2008-01-01    250
2015-01-01    250
2016-01-01    249
2017-01-01    249
2002-01-01    246
2001-01-01    245
2018-01-01    240
2000-01-01    240
1999-01-01    239
1998-01-01    237
1996-01-01    234
1997-01-01    234
1995-01-01    228
1994-01-01    225
1993-01-01    223
1992-01-01    220
1991-01-01    219
1990-01-01    208
1989-01-01    205
1988-01-01    202
1987-01-01    199


In [62]:
Wb_data.isnull().sum()

Country                                                                    0
Date                                                                       0
GDP growth (annual %)                                                      0
Adjusted savings: gross savings (% of GNI)_x                            4873
GNI growth (annual %)_x                                                 4707
Adjusted savings: energy depletion (current US$)_x                      4145
Adjusted savings: gross savings (% of GNI)_y                            4873
GNI growth (annual %)_y                                                 4707
Adjusted savings: energy depletion (current US$)_y                      4145
Adjusted savings: carbon dioxide damage (current US$)                   3788
Adjusted savings: particulate emission damage (% of GNI)                5802
Investment in energy with private participation (current US$)          10343
Unemployment, total (% of total labor force) (modeled ILO estimate)     5470

In [93]:
Wb_data = pd.merge(Wb_data, co2e_cp2000s, on = ['Country', 'Date'], how = 'outer')
Wb_data

,Country,Date,GDP growth (annual %),Adjusted savings: gross savings (% of GNI),GNI growth (annual %),Adjusted savings: energy depletion (current US$),Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: particulate emission damage (% of GNI),Investment in energy with private participation (current US$),"Unemployment, total (% of total labor force) (modeled ILO estimate)",CO2 emissions (metric tons per capita)
0,Afghanistan,2003-01-01,8.83227780370231,NaN,NaN,958155.3278581591,19983570.0004124,NaN,1580000.0,11.604000091552699,0.0504813
1,Afghanistan,2004-01-01,1.4141179802191601,NaN,NaN,1244751.78551264,16861248.694275,NaN,NaN,11.5279998779297,0.03841
2,Afghanistan,2005-01-01,11.2297148388595,NaN,NaN,1472570.39904863,24954809.424183603,NaN,NaN,11.4289999008179,0.051744
3,Afghanistan,2006-01-01,5.35740324754602,NaN,NaN,1469932.27544724,32986763.503094103,NaN,NaN,11.2239999771118,0.0624275
4,Afghanistan,2007-01-01,13.8263195390595,NaN,NaN,3237604.3216862003,47862356.5078225,NaN,NaN,11.045999526977502,0.0838928
...,...,...,...,...,...,...,...,...,...,...,...
11848,Turks and Caicos Islands,2007-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4308
11849,Turks and Caicos Islands,2008-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.7587
11850,Turks and Caicos Islands,2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6267
11851,Turks and Caicos Islands,2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.83846


## Merging IRENA & WB data:  

Uploading main dataframe to merge with the above WB features: 

In [19]:
irena = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/irena_inv.csv')

In [20]:
irena.columns

Index(['Country', 'Project', 'Date', 'Investor', 'Technology', 'Asset Class',
       'Amount (USD million)', 'Source'],
      dtype='object')

In [98]:
irena.Date = pd.to_datetime(irena.Date)

In [99]:
irena.to_csv('irena_inv.csv', index=False)

Using an 

In [100]:
iwb1 = pd.merge(irena, Wb_data, on = ['Country', 'Date'], how = 'outer')

Checking count of non-null values: 

In [101]:
iwb1_full = iwb1.dropna()
iwb1_full.shape

(2306, 17)

In [102]:
iwb1.Date.value_counts()

2017-01-01    1352
2016-01-01    1162
2014-01-01    1028
2015-01-01    1023
2013-01-01     957
2012-01-01     886
2010-01-01     846
2011-01-01     802
2009-01-01     696
2008-01-01     520
2007-01-01     476
2004-01-01     413
2006-01-01     410
2005-01-01     410
2002-01-01     362
2001-01-01     362
2003-01-01     358
2000-01-01     354
2018-01-01     240
1999-01-01     239
1998-01-01     237
1997-01-01     234
1996-01-01     234
1995-01-01     228
1994-01-01     225
1993-01-01     223
1992-01-01     220
1991-01-01     219
1990-01-01     208
1989-01-01     205
1988-01-01     202
1987-01-01     199
1986-01-01     194
1985-01-01     193
1984-01-01     190
1983-01-01     190
1982-01-01     187
1981-01-01     182
1980-01-01     171
1979-01-01     170
1978-01-01     170
1977-01-01     165
1976-01-01     164
1975-01-01     160
1973-01-01     158
1974-01-01     158
1971-01-01     158
1972-01-01     158
1970-01-01     144
1969-01-01     144
1968-01-01     140
1967-01-01     139
1966-01-01  

Retaining only data post 2000: 

In [103]:
iwb1_2000s = iwb1[iwb1.Date > '2001-01-01']

Saving to independent csv: 

In [104]:
iwb1_2000s.to_csv('Ir_wb2000s.to_csv', index=False)

In [105]:
iwb2 = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/Ir_wb2000s.to_csv')
iwb2.head()

,Country,Project,Date,Investor,Technology,Asset Class,Amount (USD million),Source,GDP growth (annual %),Adjusted savings: gross savings (% of GNI),GNI growth (annual %),Adjusted savings: energy depletion (current US$),Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: particulate emission damage (% of GNI),Investment in energy with private participation (current US$),"Unemployment, total (% of total labor force) (modeled ILO estimate)",CO2 emissions (metric tons per capita)
0,Afghanistan,HYDROELECTRIC POWER STATION I,2002-01-01,KFW,Renewable Hydropower,Grant,3.7018,2002-03 OECD Credit Report System (CRS) Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0473773238699389
1,Afghanistan,Afghanistan: Emergency National Solidarity Pro...,2003-01-01,WBG,Multiple renewables*,Grant,22.6498,2002-03 OECD Credit Report System (CRS) Database,8.83227780370231,NaN,NaN,958155.3278581591,19983570.0004124,NaN,1580000.0,11.604000091552699,0.0504813357583004
2,Afghanistan,HYDROCARBON RESOURCE SURVEY — STAGE 1,2003-01-01,Others,Renewable Hydropower,Grant,0.2698,2002-03 OECD Credit Report System (CRS) Database,8.83227780370231,NaN,NaN,958155.3278581591,19983570.0004124,NaN,1580000.0,11.604000091552699,0.0504813357583004
3,Afghanistan,SUPPORT TO THE ELECTRICITY SECTOR AND DISSEMIN...,2003-01-01,Others,Multiple renewables*,Grant,1.9594,2002-03 OECD Credit Report System (CRS) Database,8.83227780370231,NaN,NaN,958155.3278581591,19983570.0004124,NaN,1580000.0,11.604000091552699,0.0504813357583004
4,Afghanistan,Afghanistan: Emergency National Solidarity Pro...,2005-01-01,WBG,Multiple renewables*,Grant,5.9994,2004-05 OECD Credit Report System (CRS) Database,11.2297148388595,NaN,NaN,1472570.39904863,24954809.424183603,NaN,NaN,11.4289999008179,0.0517439645638815


In [ ]:
##Decided not use this data eventually due to discrepancy of total values

In [106]:
bp2 = pd.read_csv('/Users/valentinereltien/Desktop/IRENA/BP_stats2000s.csv')
bp2.head()

,Country,Date,Electricity Generation*,Hydroelectricity: Generation*,Solar_MW,WindGen_TWh,Electricity generation from other*,Electricity generation from gas*,Electricity generation from oil*,Electricity generation from coal*,"Renewables: Generation- Geothermal, Biomass and Other*",Biofuel_pd_ktoe,Carbon Dioxide Emissions
0,Australia,2014-01-01,247.6,14.5,4007.0,9.8,^,53.7,5.9,155.2,3.5,258.0,408.8
1,Australia,2016-01-01,258.5,17.7,4721.0,13.0,0.2,48.5,5.6,162.3,3.6,198.0,418.3
2,Australia,2017-01-01,259.0,13.5,5993.0,13.2,0.2,55.2,5.3,159.1,3.5,160.0,412.3
3,Australia,2018-01-01,261.4,17.3,9769.0,16.3,0.2,50.2,5.3,156.6,3.5,166.0,416.6
4,Australia,2013-01-01,249.6,19.1,3258.0,9.3,1.1,52.7,4.7,155.4,3.3,254.0,401.8


In [107]:
iwbp1 = pd.merge(iwb2, bp2, on = ['Country', 'Date'], how= 'outer')

In [108]:
renamed_cols = []
for col in iwbp1.columns:
    col = col.replace('*', '').strip()
    renamed_cols.append(col)
renamed_cols

['Country',
 'Project',
 'Date',
 'Investor',
 'Technology',
 'Asset Class',
 'Amount (USD million)',
 'Source',
 'GDP growth (annual %)',
 'Adjusted savings: gross savings (% of GNI)',
 'GNI growth (annual %)',
 'Adjusted savings: energy depletion (current US$)',
 'Adjusted savings: carbon dioxide damage (current US$)',
 'Adjusted savings: particulate emission damage (% of GNI)',
 'Investment in energy with private participation (current US$)',
 'Unemployment, total (% of total labor force) (modeled ILO estimate)',
 'CO2 emissions (metric tons per capita)',
 'Electricity Generation',
 'Hydroelectricity: Generation',
 'Solar_MW',
 'WindGen_TWh',
 'Electricity generation from other',
 'Electricity generation from gas',
 'Electricity generation from oil',
 'Electricity generation from coal',
 'Renewables: Generation- Geothermal, Biomass and Other',
 'Biofuel_pd_ktoe',
 'Carbon Dioxide Emissions']

In [109]:
iwbp1.columns = renamed_cols

In [110]:
iwbp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12026 entries, 0 to 12025
Data columns (total 28 columns):
Country                                                                12026 non-null object
Project                                                                8720 non-null object
Date                                                                   12026 non-null object
Investor                                                               8959 non-null object
Technology                                                             8959 non-null object
Asset Class                                                            8959 non-null object
Amount (USD million)                                                   8959 non-null object
Source                                                                 8959 non-null object
GDP growth (annual %)                                                  9807 non-null object
Adjusted savings: gross savings (% of GNI)                   

In [112]:
iwbp1.isnull().sum()/len(iwbp1)*100
#Consider removing CO2_ems? or replace w/ WB value? 

Country                                                                 0.000000
Project                                                                27.490437
Date                                                                    0.000000
Investor                                                               25.503077
Technology                                                             25.503077
Asset Class                                                            25.503077
Amount (USD million)                                                   25.503077
Source                                                                 25.503077
GDP growth (annual %)                                                  18.451688
Adjusted savings: gross savings (% of GNI)                             31.024447
GNI growth (annual %)                                                  30.151339
Adjusted savings: energy depletion (current US$)                       28.130717
Adjusted savings: carbon dio

In [114]:
iwbp_fl = iwbp1.dropna(thresh= )
len(iwbp_fl)

441

In [131]:
iwbp1.to_csv('Master_frame1.csv', index=False)

In [135]:
iwbp1_indexed = iwbp1.set_index('Date', drop=True)

In [136]:
iwbp1_indexed.groupby()

,Country,Project,Investor,Technology,Asset Class,Amount (USD million),Source,GDP growth (annual %),Adjusted savings: gross savings (% of GNI),GNI growth (annual %),...,Hydroelectricity: Generation,Solar_MW,WindGen_TWh,Electricity generation from other,Electricity generation from gas,Electricity generation from oil,Electricity generation from coal,"Renewables: Generation- Geothermal, Biomass and Other",Biofuel_pd_ktoe,Carbon Dioxide Emissions
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,Afghanistan,HYDROELECTRIC POWER STATION I,KFW,Renewable Hydropower,Grant,3.7018,2002-03 OECD Credit Report System (CRS) Database,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-01,Afghanistan,Afghanistan: Emergency National Solidarity Pro...,WBG,Multiple renewables*,Grant,22.6498,2002-03 OECD Credit Report System (CRS) Database,8.83227780370231,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-01,Afghanistan,HYDROCARBON RESOURCE SURVEY — STAGE 1,Others,Renewable Hydropower,Grant,0.2698,2002-03 OECD Credit Report System (CRS) Database,8.83227780370231,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-01-01,Afghanistan,SUPPORT TO THE ELECTRICITY SECTOR AND DISSEMIN...,Others,Multiple renewables*,Grant,1.9594,2002-03 OECD Credit Report System (CRS) Database,8.83227780370231,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-01,Afghanistan,Afghanistan: Emergency National Solidarity Pro...,WBG,Multiple renewables*,Grant,5.9994,2004-05 OECD Credit Report System (CRS) Database,11.2297148388595,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-01,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,246.5,23442.0,192.6,14.8,1433.9,30.4,1468.3,83.7,33849.0,5153.7
2016-01-01,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.8,34858.0,229.3,14.4,1482.1,26.0,1346.2,82.7,35986.0,5053.7
2017-01-01,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,296.8,43031.0,256.9,13.7,1394.0,23.0,1310.0,82.8,37132.0,5014.4


In [132]:
iwbp_fl.to_csv('Master_framefl.csv', index=False)

In [133]:
#Key info is Investor_Technology & Project
def dist_plotter(sample, bins=50, kde=False):
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.distplot(sample, bins=bins, kde=kde)
    plt.show()
    
dist_plotter(data.temp)
dist_plotter(data.casual)
data.season.hist()
plt.show()

TypeError: no numeric data to plot

In [119]:
iwbp1[iwbp1.Project.isnull() == False].Country.value_counts()

Brazil          763
Multilateral    393
India           365
Other Africa    296
China           218
               ... 
Malta             1
Canada            1
Lithuania         1
Cyprus            1
Fr Polynesia      1
Name: Country, Length: 191, dtype: int64

In [125]:
pj_country = pd.Series(iwbp1[iwbp1.Project.isnull() == False].Country.unique())
pj_country

0      Afghanistan
1          Albania
2          Algeria
3           Angola
4         Anguilla
          ...     
186      Venezuela
187       Viet Nam
188          Yemen
189         Zambia
190       Zimbabwe
Length: 191, dtype: object

In [128]:
others = ['EU (28)','Other Africa', 'Other Asia',
       'Other Cent Am Carib', 'Other Europe', 'Other Mid East',
       'Other Oceania', 'Other South Am']

In [130]:
pj_country.apply(lambda x: x if x not in others else None)

0      Afghanistan
1          Albania
2          Algeria
3           Angola
4         Anguilla
          ...     
186      Venezuela
187       Viet Nam
188          Yemen
189         Zambia
190       Zimbabwe
Length: 191, dtype: object

In [ ]:
# Consider removing columns or rows w/ 
# Use list of countries from other formulas? 

sns.pairplot(X, 
     vars=[col for col in X.columns if col !='iris'], 
     hue='iris', 
     diag_kind='kde')
plt.show()

Then move to Tableau

In [ ]:
[BP_2000s[col].isnull().sum() for col in BP_2000s.columns]